**DATA 2060 Final Notebook**

blah blah

In [735]:
import numpy as np

def softmax(x):
    '''
    Apply softmax to an array
    @params:
        x: the original array
    @return:
        an array with softmax applied elementwise.
    '''
    inner = np.array(x - np.max(x))
    e = np.exp(inner.astype(np.float64))
    return (e + 1e-6) / (np.sum(e) + 1e-6)

class LogisticRegression:
    '''
    Multiclass Logistic Regression that learns weights using 
    stochastic gradient descent.
    '''
    def __init__(self, n_features, n_classes, batch_size, conv_threshold):
        '''
        Initializes a LogisticRegression classifer.
        @attrs:
            n_features: the number of features in the classification problem
            n_classes: the number of classes in the classification problem
            weights: The weights of the Logistic Regression model
            alpha: The learning rate used in stochastic gradient descent
        '''
        self.n_classes = n_classes
        self.n_features = n_features
        self.weights = np.zeros((n_classes, n_features)) 
        self.alpha = 0.03  # DO NOT TUNE THIS PARAMETER
        self.batch_size = batch_size
        self.conv_threshold = conv_threshold

    def fit(self, X, Y):
        '''
        Trains the model using stochastic gradient descent
        @params:
            X: a 2D Numpy array where each row contains an example, padded by 1 column for the bias
            Y: a 1D Numpy array containing the corresponding labels for each example
        @return:
            num_epochs: integer representing the number of epochs taken to reach convergence
        '''
        #had to replace train function, should pass unit tests from HW now
        w = self.weights
        alpha = self.alpha
        b = self.batch_size
        num_epoch = 0
        converge = False
        X_and_Y = np.hstack([X,np.zeros([1,len(Y)]).T]) #so when we shuffle the labels and examples still match up
        X_and_Y[:,-1] = Y
        while converge == False:
            num_epoch += 1
            np.random.shuffle(X_and_Y) #shuffle training data, so X_and_Y is now a shuffled matrix
            shuffled_X = X_and_Y[:,:-1]
            shuffled_Y = X_and_Y[:,-1]
            last_epoch_loss = self.loss(X,Y) #calculate loss with our current weights
            for i in range(0,int(np.ceil(len(Y)/b))):
                X_batch = shuffled_X[i*b:(i+1)*b,:]
                Y_batch = shuffled_Y[i*b:(i+1)*b] #1xn array
                deriv_L_w = np.zeros(np.shape(w)) #our derivative L_w matrix
                for data_point_row in range(0,len(Y_batch)):
                    for j in range(0,self.n_classes):
                        x = X_batch[data_point_row,:]
                        if Y_batch[data_point_row] == j:
                            deriv_L_w[j,:] = deriv_L_w[j,:]+(softmax(w@x.T)[j]-1)*x
                        else:
                            deriv_L_w[j,:] = deriv_L_w[j,:]+softmax(w@x.T)[j]*x
                            
                w = w - (alpha*deriv_L_w)/len(X_batch)

            self.weights = w
            this_epoch_loss = self.loss(X,Y) #calculate loss with our new weights
            if np.abs(this_epoch_loss-last_epoch_loss) < self.conv_threshold:
                converge = True
        #print('number of epochs: ' + str(num_epoch))
        return num_epoch
        

    def loss(self, X, Y):
        '''
        Returns the total log loss on some dataset (X, Y), divided by the number of examples.
        @params:
            X: 2D Numpy array where each row contains an example, padded by 1 column for the bias
            Y: 1D Numpy array containing the corresponding labels for each example
        @return:
            A float number which is the average loss of the model on the dataset
        '''   
        # calculate loss for each example (x,y), then average them
            
        h_w_x_softmax = np.zeros([self.n_classes,len(X)])
        for i in range(0,len(X)):
            h_w_x_softmax[:,i] = self.getSoftmaxProbability(X[i,:])
        loss_sum = 0
        for j in range(0,len(Y)):
            loss_sum += -np.log(h_w_x_softmax[int(Y[j]),j])
        return loss_sum/len(Y)

        
    def getSoftmaxProbability(self, x):
        '''
        Compute softmax regular probability vector of an example belonging to each possible class.
        @params:
            x: a 1D Numpy array that is one example from the training set, padded by 1 column for the bias
        @return:
            A 1D Numpy array with one element for each possible class j, 
            where the element at j represents the probability of x belonging to j.
        '''
        return softmax(np.dot(self.weights, x))

    def predict(self, X):
        '''
        Compute predictions based on the learned weigths and examples X
        @params:
            X: a 2D Numpy array where each row contains an example, padded by 1 column for the bias
        @return:
            A 1D Numpy array with one element for each row in X containing the predicted class.
        '''
        # use one vs all algorithm for returning class with highest probablity of that x belonging to it
        # let j be each possible class

        predictions = []
        for x in X:
            normalized_prob = self.getSoftmaxProbability(x)
            # print("norm prob:", normalized_prob.shape)
            # the index of highest probablity is used as the predicted class
            x_prediction = np.argmax(normalized_prob)
            predictions.append(x_prediction)

        return np.array(predictions)

    def accuracy(self, X, Y):
        '''
        Outputs the accuracy of the trained model on a given testing dataset X and labels Y.
        @params:
            X: a 2D Numpy array where each row contains an example, padded by 1 column for the bias
            Y: a 1D Numpy array containing the corresponding labels for each example
        @return:
            a float number indicating accuracy (between 0 and 1)
        '''

        # count how many 0s (accurately predicted examples) from the difference
        predict = self.predict(X)
        diff = predict - Y
        num_correct = np.sum(diff == 0)

        accuracy = num_correct/len(Y)
        
        return accuracy

In [770]:
from sklearn.preprocessing import normalize
X_gsp_1 = np.array([[-0.5,5,1],[1,1,1],[10,2,1],[-10,-2,1],[2,1,1]])
y_gsp_1 = np.array([0,1,1,0,1])

X_gsp_1 = np.array([[-2,0,1],[-1,0,1],[-1,1,1],[0,2,1],[0,1,1],[0,-1,1],[0,-2,1],[1,0,1],[1,-1,1],[2,0,1]]) #includes bias term
y_gsp_1 = np.array([0,0,0,0,0,1,1,1,1,1])

log_reg = LogisticRegression(3,2,1,1e-4)
sk_log_reg_model = sklearn_SGDClassifier(loss = 'log_loss', fit_intercept = False, penalty = None,learning_rate = 'constant',eta0=0.03,tol=1e-4,n_iter_no_change=1)
log_reg.fit(X_gsp_1,y_gsp_1)
sk_log_reg_model.fit(X_gsp_1,y_gsp_1)

print(normalize(log_reg.weights,axis=1))
print(normalize(sk_log_reg_model.coef_))

X_2 = np.array([[-2,0,1],[-1,0,1],[-1,1,1],[0,2,1],[0,1,1],[0,-1,1],[0,-2,1],[1,0,1],[1,-1,1],[2,0,1]]) #includes bias term
y_2 = np.array([0,0,1,0,0,1,1,1,0,1])

log_reg.fit(X_2,y_2)
sk_log_reg_model.fit(X_2,y_2)
print(normalize(log_reg.weights,axis=1)[1,:])
print(normalize(sk_log_reg_model.coef_)[0])


[[-7.07110925e-01  7.07102633e-01  7.74907346e-05]
 [ 7.07110925e-01 -7.07102631e-01 -9.70796080e-05]]
[[ 7.07106554e-01 -7.07107007e-01  4.91705528e-05]]
[ 0.70896684 -0.70520629 -0.00707821]
[ 7.08533355e-01 -7.05677214e-01  3.92440976e-04]


In [797]:
### Unit Tests LogisticRegression ###
import pytest
from sklearn.linear_model import LogisticRegression as SKLogisticRegression
from sklearn.linear_model import SGDClassifier as SKSGDClassifier
from sklearn.preprocessing import normalize

np.random.seed(42)

#softmax function, calculated results by hand
softmax_test_1 = np.array([1,2,3,4,5]) #generic example
assert softmax(softmax_test_1) == pytest.approx([0.01166,0.03168,0.08612,0.23412,0.6364086], .001)
softmax_test_2 = np.array([-2,0,3,-2,3]) #tests if can handle multiple of the same values/max value occurs multiple times
assert softmax(softmax_test_2) == pytest.approx([0.003266,0.024131,0.484669,0.003266,0.484669], .001)


#weights during fit
#training points can be split by a halfspace, 
X_weights_1 = np.array([[-2,0,1],[-1,0,1],[-1,1,1],[0,2,1],[0,1,1],[0,-1,1],[0,-2,1],[1,0,1],[1,-1,1],[2,0,1]]) #includes bias term
y_weights_1 = np.array([0,0,0,0,0,1,1,1,1,1])

#the training points cannot be split by a halfspace, at least 2 training points will have to be mislabeled
X_weights_2 = np.array([[-2,0,1],[-1,0,1],[-1,1,1],[0,2,1],[0,1,1],[0,-1,1],[0,-2,1],[1,0,1],[1,-1,1],[2,0,1]]) #includes bias term
y_weights_2 = np.array([0,0,1,0,0,1,1,1,0,1])

own_log_reg = LogisticRegression(3,2,1,1e-4)
#penalty = None means no regularization, learning rate = eta0 is our alpha in LogisticRegression, tolerance = tol set to our convergence threshold, n_iter_no_change = 1 means if we hit convergence once to stop
sk_sgd_log_reg = SKSGDClassifier(loss = 'log_loss', fit_intercept = False, penalty = None,learning_rate = 'constant',eta0=0.03,tol=1e-4,n_iter_no_change=1)

#first check shape of weights
assert (own_log_reg.weights.shape == np.array([2,3])).all()

#check for each of these datasets, if the normalized weights for each model are within 0.01 of each other for each feature
own_log_reg.fit(X_weights_1,y_weights_1)
sk_sgd_log_reg.fit(X_weights_1,y_weights_1)
assert (abs(normalize(own_log_reg.weights,axis=1)[1] - normalize(sk_sgd_log_reg.coef_)[0]) <= 0.01).all()

own_log_reg.fit(X_weights_2,y_weights_2)
sk_sgd_log_reg.fit(X_weights_2,y_weights_2)
assert (abs(normalize(own_log_reg.weights,axis=1)[1] - normalize(sk_sgd_log_reg.coef_)[0]) <= 0.01).all()
        


#### loss function ###
log_reg = LogisticRegression(3,2,1,1e-4)
log_reg.weights = np.array([[-0.5,2,0.1],[0.5,-2,-0.1]]) #set weights to some random 2 class, 3 features (including bias) array
#test on dataset with one datapoint
#case with vector of all 0's, should be same loss for both classes
X_loss_1 = np.array([[0,0,0]])
assert log_reg.loss(X_loss_1,np.array([0])) == pytest.approx(np.array([0.6931472]),0.001) #-ln(0.5)
assert log_reg.loss(X_loss_1,np.array([1])) == pytest.approx(np.array([0.6931472]),0.001)

#with multiple datapoints, random test
X_loss_2 = np.array([[-1,2,0.3],[1.5,-0.1,2]])
y_loss_2 = np.array([0,1])
assert log_reg.loss(X_loss_2,y_loss_2) == pytest.approx(0.10076465,0.001)



### getSoftmaxProbability function ###
X_gsp_1 = np.array([[-0.5,5,1],[1,1,1],[10,2,1],[-10,-2,1],[2,1,1]]) #random test data with bias included
y_gsp_1 = np.array([0,1,1,0,1])

log_reg = LogisticRegression(3,2,1,1e-4)
log_reg.fit(X_gsp_1,y_gsp_1)
test_gsp_1 = np.array([0,0,0])

#no matter our weights, if datapoint is vector of 0's, getSoftmaxProbability should give us around an equal probability for each class
assert log_reg.getSoftmaxProbability(test_gsp_1) == pytest.approx(np.array([0.5,0.5]),0.001) 
#set weights to some random 2 class, 3 features (including bias) array, test on random datapoint
log_reg.weights = np.array([[-0.5,2,0.1],[0.5,-2,-0.1]])
test_gsp_2 = np.array([3,-0.9,1.3])
assert log_reg.getSoftmaxProbability(test_gsp_2) == pytest.approx(np.array([0.0017612,0.9982388]),0.001) #calculated by hand



### predict function ###
#the training points can be split by a halfspace
X_1 = np.array([[-2,0,1],[-1,0,1],[-1,1,1],[0,2,1],[0,1,1],[0,-1,1],[0,-2,1],[1,0,1],[1,-1,1],[2,0,1]]) #includes bias term
y_1 = np.array([0,0,0,0,0,1,1,1,1,1])
X_test_1 = np.array([[0,5,1],[-10,1,1],[-2,2,1],[2,-2,1],[1,-10,1],[5,0,1]]) #includes terms that would be right on the edge of a correct halfspace

#the training points cannot be split by a halfspace, at least 2 training points will have to be mislabeled
X_2 = np.array([[-2,0,1],[-1,0,1],[-1,1,1],[0,2,1],[0,1,1],[0,-1,1],[0,-2,1],[1,0,1],[1,-1,1],[2,0,1]]) #includes bias term
y_2 = np.array([0,0,1,0,0,1,1,1,0,1])
X_test_2 = np.array([[0,5,1],[-10,1,1],[-2,2,1],[2,-2,1],[1,-10,1],[5,0,1]]) #includes terms that would be right on the edge of a correct halfspace

#uneven number of training points for each label
X_3 = np.array([[-2,0,1],[-1,1,1],[0,2,1],[0,-1,1],[0,-2,1],[1,0,1],[1,-1,1],[2,0,1]]) #includes bias term
y_3 = np.array([0,0,0,1,1,1,1,1])
X_test_3 = np.array([[0,5,1],[-10,1,1],[-2,2,1],[2,-2,1],[1,-10,1],[5,0,1]]) #includes terms that would be right on the edge of a correct halfspace

#initialize LogisticRegression model with n_classes = 2, batch_size = 1, conv_threshold = 1e-4
log_reg = LogisticRegression(3,2,1,1e-4)

#train our LogisticRegression model on each training dataset, then test predictions
log_reg.fit(X_1,y_1)
assert (log_reg.predict(X_test_1) == np.array([0,0,0,1,1,1])).all()

log_reg.fit(X_2,y_2)
assert (log_reg.predict(X_test_2) == np.array([0,0,0,1,1,1])).all()

log_reg.fit(X_3,y_3)
assert (log_reg.predict(X_test_3) == np.array([0,0,0,1,1,1])).all()



### accuracy function ###
#test accuracy based on previous prediction tests, we know prediction should be [0,0,0,1,1,1]
accuracy_test_1 = np.array([1,1,1,0,0,0]) #make sure labeling is correct (flip labels, get 0 accuracy)
accuracy_test_2 = np.array([1,0,0,1,0,1]) #make sure percentage for accuracy is correct to the nearest 0.001 for long decimals
log_reg.fit(X_1,y_1)
assert log_reg.accuracy(X_test_1, accuracy_test_1) == 0
assert log_reg.accuracy(X_test_1, accuracy_test_2) == pytest.approx(0.667,0.001)

print('Passed all tests!')

Passed all tests!


## Attempted unit tests for Logistic Regression class and softmax functions

In [703]:
import pytest
from sklearn.linear_model import LogisticRegression as SKLogisticRegression
from sklearn.linear_model import SGDClassifier as sklearn_SGDClassifier
#test for softmax, calculated results by hand
softmax_test_1 = np.array([1,2,3,4,5]) #generic example
assert softmax(softmax_test_1) == pytest.approx([0.01166,0.03168,0.08612,0.23412,0.6364086], .001)
softmax_test_2 = np.array([-2,0,3,-2,3]) #tests if can handle multiple of the same values/max value occurs multiple times
assert softmax(softmax_test_2) == pytest.approx([0.003266,0.024131,0.484669,0.003266,0.484669], .001)

random.seed(0)
np.random.seed(0)
#test for our logistic regression function with that from sklearn
X_1 = np.array([[1,2],[2.3,6,],[-9,0],[-1,2.34]]) #generic example
bias_1 = np.ones((len(X_1),1))
X_1 = np.hstack((X_1, bias_1))
Y_1 = np.array([1,1,0,0]) #binary

X_test_1 = np.array([[3,4,1],[-90,1,1],[3.2,-2.2,1],[-22,4.3,1]])


batch_size = 4
conv_thresh = 0.01
own_log_reg_model = LogisticRegression(len(X_1[0,:]),2,batch_size,conv_thresh) #num_features does not include bias, our own model
own_log_reg_model.fit(X_1,Y_1)
sklearn_log_reg_model = SKLogisticRegression(fit_intercept = False,tol=conv_thresh,C=1) #sklearn LogisticRegression model
sklearn_log_reg_model.fit(X_1,Y_1)
sklearn_SGD_model = sklearn_SGDClassifier(loss='log_loss',fit_intercept = False,alpha=0.03,tol=conv_thresh,n_iter_no_change=1, random_state = 0) #sklearn SGDClassifier model
#sklearn_SGD_model.fit(X_1,Y_1)

#test weights
#print(own_log_reg_model.weights)
#print(sklearn_log_reg_model.coef_)
#print(sklearn_SGD_model.coef_)

#test predictions
#print(own_log_reg_model.predict(X_test_1)) #appears predictions match, weights are a bit off
#print(sklearn_log_reg_model.predict(X_test_1))




#no sklearn method to use batch stochastic gradient descent
batch_size = 1
conv_thresh = 0.0001
#sk_log_reg_model = SKLogisticRegression(fit_intercept = False,tol=conv_thresh,penalty='l2',C=1)
regularized = RegularizedLogisticRegression(batch_size)
#learning rate is alpha? alpha is lambda if we do reg
sk_log_reg_model = sklearn_SGDClassifier(loss = 'log_loss', fit_intercept = False, penalty = None,learning_rate = 'constant',eta0=0.03,tol=conv_thresh,n_iter_no_change=1, random_state = 0)
# Smaller set of train and test data
X_1 = np.random.rand(10, 2)
X_1 = np.hstack((X_1,np.ones((len(X_1),1))))
y_1 = np.random.randint(2,size=10)
if y_1.sum() == len(y_1):
    y_1[0:int(len(y_1)/2)] = 0
elif y_1.sum() == 0:
    y_1[0:int(len(y_1)/2)] = 1
X_test_1 = np.random.rand(5, 2)
X_test_1 = np.hstack((X_test_1,np.ones((len(X_test_1),1))))

own_log_reg_model_1 = LogisticRegression(len(X_1[0,:]),2,batch_size,conv_thresh)
regularized.train(X_1,y_1)
own_log_reg_model_1.fit(X_1,y_1)
sk_log_reg_model.fit(X_1,y_1)

print('own log reg predictions:')
print(own_log_reg_model_1.predict(X_test_1))
print('sklearn log reg predictions:')
print(sk_log_reg_model.predict(X_test_1))

print(own_log_reg_model_1.weights)
print(sk_log_reg_model.coef_)
#print(regularized.predict(X_test_1))
#assert(1-np.sum(np.abs(own_log_reg_model_1.predict(X_test_1)-sk_log_reg_model.predict(X_test_1)))/len(X_test_1) >= 0.8)
#assert (own_log_reg_model_1.predict(X_test_1) == sk_log_reg_model.predict(X_test_1)).all()

# Larger set of train and test data
X_2 = np.random.rand(40, 5)
X_2 = np.hstack((X_2,np.ones((len(X_2),1))))
y_2 = np.random.randint(2, size = 40)
if y_2.sum() >= len(y_2)*0.95:
    y_2[0:len(y_2)/2] = 0
elif y_1.sum() == 0:
    y_2[0:len(y_2)/2] = 1
X_test_2 = np.random.rand(10, 5)
X_test_2 = np.hstack((X_test_2,np.ones((len(X_test_2),1))))

own_log_reg_model_2 = LogisticRegression(len(X_2[0,:]),2,batch_size,conv_thresh)
regularized.train(X_2,y_2)
own_log_reg_model_2.fit(X_2,y_2)
sk_log_reg_model.fit(X_2,y_2)

print(own_log_reg_model_2.predict(X_test_2))
print(sk_log_reg_model.predict(X_test_2))

print(own_log_reg_model_2.weights)
print(sk_log_reg_model.coef_)
#print(regularized.predict(X_test_2))
#assert(1-np.sum(np.abs(own_log_reg_model_2.predict(X_test_2)-sk_log_reg_model.predict(X_test_2)))/len(X_test_2) >= 0.8)
#assert (own_log_reg_model_2.predict(X_test_2) == sk_log_reg_model.predict(X_test_2)).all()


# Larger set of train and test data with wider range of values 
#actually don't do this bc we want to normalize the data!
X_3 = np.random.uniform(low=-15, high=15, size=(30,5))
X_3 = np.hstack((X_3,np.ones((len(X_3),1))))
y_3 = np.random.randint(2, size = 30)
X_test_3 = np.random.uniform(low=-15, high=15, size=(5,5))
X_test_3 = np.hstack((X_test_3,np.ones((len(X_test_3),1))))

own_log_reg_model_3 = LogisticRegression(len(X_3[0,:]),2,batch_size,conv_thresh)
regularized.train(X_3,y_3)
own_log_reg_model_3.fit(X_3,y_3)
sk_log_reg_model.fit(X_3,y_3)

print(own_log_reg_model_3.predict(X_test_3))
print(sk_log_reg_model.predict(X_test_3))
print(regularized.predict(X_test_3))
#assert(1-np.sum(np.abs(own_log_reg_model_3.predict(X_test_3)-sk_log_reg_model.predict(X_test_3)))/len(X_test_3) >= 0.8)
#assert (own_log_reg_model_3.predict(X_test_3) == sk_log_reg_model.predict(X_test_3)).all()



own log reg predictions:
[1 1 0 1 0]
sklearn log reg predictions:
[1 1 0 1 0]
[[ 0.93643481 -1.41140007  0.12291163]
 [-0.93646024  1.41138718 -0.12294918]]
[[-1.63431988  2.4166285  -0.23592541]]
[1 1 1 1 0 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[[-0.66010379  0.24330464  0.67263192  0.61904541 -0.16628908 -0.57664825]
 [ 0.66008693 -0.24332256 -0.67264786 -0.61906199  0.16627398  0.57661731]]
[[ 0.40167959 -0.08675389 -0.29310007 -0.34243455  0.09834801  0.22974414]]
[0 1 0 1 1]
[0 1 1 1 0]
[0. 0. 0. 0. 1.]


In [337]:
random.seed(0)
np.random.seed(0)
train_1 = np.array([[1,2,1],[3,-2,1],[3,3,1],[-4,1,1],[-1,-2,1],[-3,2,1]])
y_train_1 = np.array([1,1,1,0,0,0])
own_log = LogisticRegression(3,2,batch_size,conv_thresh)
SK_log = sklearn_SGDClassifier(loss = 'log_loss', fit_intercept = False, penalty = None,learning_rate = 'constant',eta0=0.03,tol=conv_thresh,n_iter_no_change=1)
own_log.train(train_1,y_train_1)
SK_log.fit(train_1,y_train_1)

print(own_log.weights)
print(SK_log.coef_)

test_1 = np.array([[1,2,1],[-4,1,1],[-10,-20,1],[200,-100,1]])
print(own_log.predict(test_1))
print(SK_log.predict(test_1))


AttributeError: 'LogisticRegression' object has no attribute 'train'

## Unit Tests for Logistic Regression from HW3

In [738]:
import random
random.seed(0)
np.random.seed(0)

# Creates Test Model with 2 predictors, 2 classes, a Batch Size of 5 and a Threshold of 1e-2 (used 3 for predictors to include bias)
test_model1 = LogisticRegression(3, 2, 5, 1e-2)

# Creates Test Data
x_bias = np.array([[0,4,1], [0,3,1], [5,0,1], [4,1,1], [0,5,1]])
y = np.array([0,0,1,1,0])
x_bias_test = np.array([[0,0,1], [-5,3,1], [9,0,1], [1,0,1], [6,-7,1]])
y_test = np.array([0,0,1,0,1])

# Creates Test Model with 2 predictors, 1 classes, a Batch Size of 1 and a Threshold of 1e-2
test_model2 = LogisticRegression(3, 3, 1, 1e-2)

# Creates Test Data
x_bias2 = np.array([[0,0,1], [0,3,1], [4,0,1], [6,1,1], [0,1,1], [0,4,1]])
y2 = np.array([0,1,2,2,0,1])
x_bias_test2 = np.array([[0,0,1], [-5,3,1], [9,0,1], [1,0,1]])
y_test2 = np.array([0,1,2,0])


# Test Model Loss
print(test_model1.loss(x_bias,y))
assert test_model1.loss(x_bias, y) == pytest.approx(0.693, .001) # Checks if answer is within .001
assert test_model2.loss(x_bias2, y2) == pytest.approx(1.099, .001) # Checks if answer is within .001

# Test Train Model and Checks Model Weights
assert test_model1.fit(x_bias, y) == 14
assert test_model1.weights == pytest.approx(np.array([[-0.218, 0.231, 0.0174], [ 0.218, -0.231, -0.0174]]), 0.01) # Answer within .01

assert test_model2.fit(x_bias, y) == 9
assert test_model2.weights == pytest.approx(np.array([[-0.300,  0.560,  0.093], [ 0.523, -0.257,  0.032], [-0.226, -0.304, -0.123]]), .05)

# Test Model Predict
assert (test_model1.predict(x_bias_test) == np.array([0., 0., 1., 1., 1.])).all()
assert (test_model2.predict(x_bias_test2) == np.array([0, 0, 1, 1])).all()

# Test Model Accuracy
assert test_model1.accuracy(x_bias_test, y_test) == .8
assert test_model2.accuracy(x_bias_test2, y_test2) == .25

0.6931466805603205


## Code for OnevsAll and AllPairs

In [694]:
### One vs. All Logistic Regression multi-class regression classifier ###
import numpy as np
import itertools as it

class OneVsAll:
    def __init__(
        self,
        n_features: int,
        n_classes: int,
        batch_size: int,
        conv_threshold: float,
        classifier = 'own',
    ) -> None:
        self.n_classes = n_classes
        self.classifier = classifier
        if classifier == 'own':
            self.models = [
                LogisticRegression(n_features, 2, batch_size, conv_threshold) 
                for _ in range(n_classes)
            ]
        else:
            self.models = [
                SKLogisticRegression(fit_intercept = False) 
                for _ in range(n_classes)
            ]

    def train(self, X: np.ndarray, Y: np.ndarray) -> None:
        # Train a binary classifier for each class against all others
        for class_label in range(self.n_classes):
            binary_labels = (Y == class_label).astype(int)
            if self.classifier == 'own':
                self.models[class_label].train(X, binary_labels)
            else:
                self.models[class_label].fit(X, binary_labels)

    def predict(self, X: np.ndarray) -> np.ndarray:
        # Get the probabilities of each class for each data point
        # print("X", X.shape)

        predictions = np.zeros(len(X))
        if self.classifier == 'own':
            for j in range(0,len(X)): #for each datapoint
                probabilities = np.zeros(len(self.models))
                for i in range(0,len(self.models)):
                    #for each model i, it trains on if we relabel current class i as '1' and all other classes as '0'
                    #so, the value from getSoftmaxProbability at index 1 would be probability of being in class 1 for that model, or our current class i
                    prob_x_class_i = self.models[i].getSoftmaxProbability(X[j,:])[1] 
                    probabilities[i] = prob_x_class_i
                predictions[j] = np.argmax(probabilities) #for all classes get the class with greatest probability
        else:
            for j in range(0,len(X)): #for each datapoint
                probabilities = np.zeros(len(self.models))
                for i in range(0,len(self.models)):
                    #same as our own function except we construct our weights for both classes instead of just one class like from sklearn's coef_
                    weights = np.zeros([2,len(self.models[i].coef_[0])])
                    weights[0,:] = -self.models[i].coef_[0]
                    weights[1,:] = self.models[i].coef_[0]
                    prob_x_class_i = softmax(np.dot(weights,(X[j,:])))[1]
                    probabilities[i] = prob_x_class_i
                predictions[j] = np.argmax(probabilities) #for all classes get the class with greatest probability
            
        return predictions

    def accuracy(self, X: np.ndarray, Y: np.ndarray) -> float:
        predictions = self.predict(X)
        return np.mean(predictions == Y)


In [672]:
### All Pairs Logistic Regression multi-class regression classifier ###

class AllPairs:
    def __init__(
        self,
        n_features: int,
        n_classes: int,
        batch_size: int,
        conv_threshold: float,
        classifier: str = "own",
    ):
        self.n_classes = n_classes
        self.pairs = list(it.combinations(range(n_classes), 2))
        print(self.pairs)
        self.classifier = classifier
        if classifier == "own":
            self.models = {
                (i, j): LogisticRegression(
                    n_features,
                    2,
                    batch_size,
                    conv_threshold
                )
                for i, j in self.pairs
            }
        else:
            self.models = {
                (i, j): SKLogisticRegression(
                    fit_intercept=False
                )
                for i, j in self.pairs
            }

    def train(self, X: np.ndarray, Y: np.ndarray) -> None:
        # Iterate over all pair combinations
        #print("self pairs:", self.pairs)
        for i, j in self.pairs:
            #print("ij", i,j)
            indices = (Y == i) | (Y == j)

            # Get appropriate data
            X_pair, Y_pair = X[indices], Y[indices]
            Y_pair = (Y_pair == j).astype(int)

            # Train on the pair
            if self.classifier == 'own':
                self.models[(i, j)].train(X_pair, Y_pair)
            else:
                self.models[(i, j)].fit(X_pair, Y_pair)

    def predict(self, X: np.ndarray) -> np.ndarray:
        # Count predictions for each class
        votes = np.zeros((len(X), self.n_classes))
        for (i, j), model in self.models.items():
            predictions = np.array(model.predict(X))
            # print("predictions shape:", predictions.shape, predictions)
            votes[:, j] += predictions
            votes[:, i] += (1 - predictions)

        # Return the class with the most predictions
        return np.argmax(votes, axis=1)

    def accuracy(self, X: np.ndarray, Y: np.ndarray) -> float:
        predictions = self.predict(X)
        return np.mean(predictions == Y)


In [657]:
### Dummy test ###
n_samples = 500
n_features = 10
n_classes = 5
X = np.random.rand(n_samples, n_features)
Y = np.random.randint(0, n_classes, n_samples)

#One-vs-All
ova = OneVsAll(n_features, n_classes, batch_size=32, conv_threshold=1e-4)
ova.train(X, Y)
print(f"One-vs-All Accuracy: {ova.accuracy(X, Y)}")

# All-Pairs
all_pairs = AllPairs(n_features, n_classes, batch_size=32, conv_threshold=1e-4)
all_pairs.train(X, Y)
print(f"All-Pairs Accuracy: {all_pairs.accuracy(X, Y)}")

# TODO: Add unit tests


One-vs-All Accuracy: 0.264
[(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
All-Pairs Accuracy: 0.258


In [670]:
# Run SKLearn's one verse all logistic regression using the example X_1 and X_2 datasets.
# Get the weight matrix after training for each of the sub binary classifiers to
# compare against the weights of our own implementation of one verse all
# logistic regression. Also compare predictions and accuracy.
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

# Set the random seed
np.random.seed(42)

# Smaller test data
X_1 = np.random.rand(6, 3)
y_1 = np.array([0, 1, 2])
X_test_1 = np.random.rand(3, 3)

# Larger test data
X_2 = np.random.rand(8, 4)
y_2 = np.array([0, 1, 2, 0, 1, 2, 0, 2])
X_test_2 = np.random.rand(4, 4)

# Train sklearn all pairs logistic regression
sk_model = OneVsOneClassifier(SKLogisticRegression(fit_intercept=False))
sk_model.fit(X_2, y_2)

# Print weights of each classes binary classification model
print("SKLearn weights:")
print(sk_model.estimators_)
for i, model in enumerate(sk_model.estimators_):
    print(f"Class {i} weights {model.coef_}")
    print()


# Train our own all pairs logistic regression
all_pairs = AllPairs(4, 3, 1, 1e-4, "sk")
all_pairs.train(X_2, y_2)

# Print weights of each classes binary classification model
print("Our weights:")
for i, model in all_pairs.models.items():
    print(f"Class {i} weights {model.coef_}")
    print()

SKLearn weights:
(LogisticRegression(fit_intercept=False), LogisticRegression(fit_intercept=False), LogisticRegression(fit_intercept=False))
Class 0 weights [[-0.17315305 -0.51387022  0.16819876  0.09515732]]

Class 1 weights [[ 0.24780742 -0.31339288 -0.40551825  0.21331106]]

Class 2 weights [[ 0.38846112  0.22371409 -0.54569867  0.1243528 ]]

[(0, 1), (0, 2), (1, 2)]
Our weights:
Class (0, 1) weights [[ 0.17315305  0.51387022 -0.16819876 -0.09515732]]

Class (0, 2) weights [[-0.24780742  0.31339288  0.40551825 -0.21331106]]

Class (1, 2) weights [[-0.38846112 -0.22371409  0.54569867 -0.1243528 ]]



## German Numerical Credit Data

In [695]:
import pandas as pd

def get_credit():
    """
    Gets and preprocesses German Credit data
    """
    #data = pd.read_csv('./data/german_numerical-binsensitive.csv') # Reads file - may change
    data = pd.read_csv('./german_numerical-binsensitive.csv')
    # MONTH categorizing
    data['month'] = pd.cut(data['month'],3, labels=['month_1', 'month_2', 'month_3'], retbins=True)[0]
    # month bins: [ 3.932     , 26.66666667, 49.33333333, 72.        ]
    a = pd.get_dummies(data['month'])
    data = pd.concat([data, a], axis = 1)
    data = data.drop(['month'], axis=1)

    # CREDIT categorizing
    data['credit_amount'] = pd.cut(data['credit_amount'], 3, labels=['cred_amt_1', 'cred_amt_2', 'cred_amt_3'], retbins=True)[0]
    # credit bins: [  231.826,  6308.   , 12366.   , 18424.   ]
    a = pd.get_dummies(data['credit_amount'])
    data = pd.concat([data, a], axis = 1)
    data = data.drop(['credit_amount'], axis=1)

    for header in ['investment_as_income_percentage', 'residence_since', 'number_of_credits']:
        a = pd.get_dummies(data[header], prefix=header)
        data = pd.concat([data, a], axis = 1)
        data = data.drop([header], axis=1)

    # change from 1-2 classes to 0-1 classes
    data['people_liable_for'] = data['people_liable_for'] -1
    data['credit'] = -1*(data['credit']) + 2 # original encoding 1: good, 2: bad. we switch to 1: good, 0: bad

    # balance dataset
    data = data.reindex(np.random.permutation(data.index)) # shuffle
    pos = data.loc[data['credit'] == 1]
    neg = data.loc[data['credit'] == 0][:350]
    combined = pd.concat([pos, neg])

    y = data.iloc[:, data.columns == 'credit'].to_numpy()
    x = data.drop(['credit', 'sex', 'age', 'sex-age'], axis=1).to_numpy()

    # split into train and validation
    X_train, X_val, y_train, y_val = x[:350, :], x[351:526, :], y[:350, :].reshape([350,]), y[351:526, :].reshape([175,])

    # keep info about sex and age of validation rows for fairness portion
    x_sex = data.iloc[:, data.columns == 'sex'].to_numpy()[351:526].reshape([175,])
    x_age = data.iloc[:, data.columns == 'age'].to_numpy()[351:526].reshape([175,])
    x_sex_age = data.iloc[:, data.columns == 'sex-age'].to_numpy()[351:526].reshape([175,])

    return X_train, X_val, y_train, y_val, x_sex, x_age, x_sex_age

np.random.seed(0)
X_train, X_test, y_train, y_test, x_sex, x_age, x_sex_age = get_credit()
n_samples = len(X_train)
n_features = len(X_train[0])+1
n_classes = 2

# process data
X_train[X_train == False] = 0.0
X_train[X_train == True] = 1.0
X_test[X_test == False] = 0.0
X_test[X_test == True] = 1.0

train_bias = np.ones((n_samples, 1))
X_train = np.hstack((X_train, train_bias))

test_bias = np.ones((len(X_test), 1))
X_test = np.hstack((X_test, test_bias))

print("x shape:", X_train.shape, X_test.shape)
print("num features:", n_features)

batch_size = 50
conv_threshold = 1e-5

# One-vs-All
ova = OneVsAll(n_features, n_classes, batch_size=batch_size, conv_threshold=conv_threshold)
ova.train(X_train, y_train)
print(f"One-vs-All Accuracy: {ova.accuracy(X_test, y_test)}")

ova = OneVsAll(n_features, n_classes, batch_size=batch_size, conv_threshold=conv_threshold, classifier = 'sk')
ova.train(X_train, y_train)
print(f"One-vs-All Accuracy: {ova.accuracy(X_test, y_test)}")

# All-Pairs

all_pairs = AllPairs(n_features, n_classes, batch_size=batch_size, conv_threshold=conv_threshold)
all_pairs.train(X_train, y_train)
print(f"All-Pairs Accuracy: {all_pairs.accuracy(X_test, y_test)}")


x shape: (350, 70) (175, 70)
num features: 70
One-vs-All Accuracy: 0.7028571428571428
One-vs-All Accuracy: 0.7028571428571428
[(0, 1)]
All-Pairs Accuracy: 0.7028571428571428


## SkLearn Iris Data

In [673]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler

data = load_iris()
X = data.data
Y = data.target

#normalize the data to value between 0 and 1 for each attribute
scaler = MinMaxScaler()
scaler.fit(X)
#X = scaler.transform(X)


n_samples = len(X)
n_features = len(X[0])+1 #for bias
n_classes = 3
batch_size = 1
conv_threshold = 1e-4

print("Number samples: ", n_samples, "Number Feats: ", n_features)

# shuffle data
np.random.seed(1)
p = np.random.permutation(n_samples)
X_ = X[p]
Y_ = Y[p]

# add bias to shuffled data
bias = np.ones((n_samples,1))
X_ = np.hstack((X_, bias))

# split data 80/20
train_size = int(.8*n_samples)
X_train = X_[:train_size,:]
y_train = Y_[:train_size]

X_test = X_[train_size:, :]
y_test = Y_[train_size:]

### our own model accuracy ###
# One-vs-All
ova = OneVsAll(n_features, n_classes, batch_size=batch_size, conv_threshold=conv_threshold)
ova.train(X_train, y_train)
print(f"One-vs-All Accuracy: {ova.accuracy(X_test, y_test)}")
print(ova.predict(X_test))

# All-Pairs

all_pairs = AllPairs(n_features, n_classes, batch_size=batch_size, conv_threshold=conv_threshold)
all_pairs.train(X_train, y_train)
print(f"All-Pairs Accuracy: {all_pairs.accuracy(X_test, y_test)}")
print(all_pairs.predict(X_test))



Number samples:  150 Number Feats:  5
One-vs-All Accuracy: 0.9
[0. 2. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 2. 0. 0. 1. 1. 1.
 1. 1. 2. 1. 2. 0.]
[(0, 1), (0, 2), (1, 2)]
All-Pairs Accuracy: 0.9333333333333333
[0 2 0 1 0 1 1 0 0 1 0 1 1 0 1 1 2 1 2 0 0 2 1 2 1 2 2 2 2 0]


## Using sklearn model on Iris Dataset (can change between LogisticRegression and SGDClassifier as binary algo)

In [675]:
### using sklearn model get accuracy ### there's some randomness with the sklearn function because of SGD!!
#from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

#sk_model_allpairs = OneVsOneClassifier(SKLogisticRegression(fit_intercept=False))
#sk_model_ovr = OneVsRestClassifier(SKLogisticRegression(fit_intercept=False))
sk_model_allpairs = OneVsOneClassifier(sklearn_SGDClassifier(loss = 'log_loss', fit_intercept = False, penalty = None,learning_rate = 'constant',eta0=0.03,tol=conv_threshold,n_iter_no_change=1))
sk_model_ovr = OneVsRestClassifier(sklearn_SGDClassifier(loss = 'log_loss', fit_intercept = False, penalty = None,learning_rate = 'constant',eta0=0.03,tol=conv_threshold,n_iter_no_change=1))

sk_model_allpairs.fit(X_train, y_train)
sk_model_ovr.fit(X_train,y_train)

print(f"sk model one-vs-all accuracy: {sk_model_ovr.score(X_test,y_test)}")
print(sk_model_ovr.predict(X_test))
print(f"sk model all-pairs accuracy: {sk_model_allpairs.score(X_test,y_test)}")
print(sk_model_allpairs.predict(X_test))


###for raw data accuracy works on seed 30, 54, conv_threshold = 1e-4
###for normalized data accuracy works on seed 107, 170,174, conv_threshold = 1e-5
#comparing own models with own logistic regression against classifiers with SGDClassifier as their baseline log reg






sk model one-vs-all accuracy: 0.9
[0 2 0 1 0 1 1 0 0 1 0 1 2 0 1 1 2 1 2 0 0 2 1 2 1 2 2 2 2 0]
sk model all-pairs accuracy: 0.9333333333333333
[0 2 0 1 0 1 1 0 0 1 0 1 1 0 1 1 2 1 2 0 0 2 1 2 1 2 2 2 2 0]


In [655]:
#we got 0.766666 accuracy and 0.9333333 accuracy for ova and all pairs for our implementation
#0.7, 0.9 accuracy for sklearn implementation for the below datasets! if set seed = 0
#get better results for our model too if seed = 6 (all conv_threshold = 1e-4, batch_size=1)

#for seed = 30 we get same accuracy for both models! for raw data


    

In [691]:
print(len(sk_model_ovr.estimators_[0].coef_[0]))

5
